<a href="https://colab.research.google.com/github/hawkmak3r/NLP/blob/master/0889960_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
#libraries we need to compute this assignment
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.model_selection import train_test_split
import numpy as np
import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [65]:
#importing for data into our system
#please change your path according your use
dataset = pd.read_csv("train.tsv",delimiter='\t',encoding='utf-8')
dataset.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [66]:
#this return a random sample of items from an axis of object.
dataset = dataset.sample(frac=1)
#as we can see above we have 2 index which typically happens when dealing with
dataset = dataset.reset_index(drop=True)
dataset.head()
#this way we are generating random

,PhraseId,SentenceId,Phrase,Sentiment
0,65341,3308,is historical filmmaking without the balm of r...,3
1,49802,2442,the more outrageous bits achieve a shock-you-i...,3
2,36259,1714,"Give Shapiro , Goldman , and Bolado credit for...",1
3,16041,689,petty,1
4,91001,4734,top and movies,2


In [0]:
#as per splitting our data 70 & 30 as per given in our assigment
X_train, X_test, Y_train, Y_test = train_test_split(dataset ['Phrase'], dataset ['Sentiment'], test_size=0.3)
words=[]
#passing our dataframe into list so we can manipulate it with easy
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())


In [0]:
#for training we need to tokenize our data 
#tokenize help us to train our data
#which will be stored in words[] list

for i in range(len(X_train)):
  words.append([list(word_tokenize(X_train[i])), Y_train[i]]) 
X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  words.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

In [0]:
#now time to preprocess our data
#here we are using proterStemmmer 
#then word lemmatizer for pre process
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
remove_stopwords = True
#as we have in build libraries for removing stopwords adn puncation
#we are going to be using that to help us with better
removePuncs = True
useStemming = False
useLemma = False



In [70]:
#this loop will itrate over words and as per labels and starts removing 
#no neccasary stuff
for l in range(len(words)):                    
  label = words[l][1]                        
  tmpReview = []                                  
  for w in words[l][0]:                       
    newWord = w                                   
    if remove_stopwords and (w in stopwords_en):  
      continue                                    
    if removePuncs and (w in punctuations):       
      continue                                    
    if useStemming:
      newWord = lancaster.stem(newWord) 
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                     
  words[l] = (tmpReview, label)              
  words[l] = (' '.join(tmpReview), label) 

print(words[0])


('natural-seeming actors', 3)


In [71]:
#ONCE WE OUR WORDS WE WILL BE ADDING IT AND WILL MAKE A DATASET
#WITHOUT ANY repitation 
dataset = pd.DataFrame(words, columns=['text', 'sentiment']) 
dataset.head()

,text,sentiment
0,natural-seeming actors,3
1,gorgeous film must everyone junior scientists ...,4
2,canny derivative wildly gruesome,3
3,cutesy film references,2
4,could confusing horrifying vision intense engr...,1


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['text'],  dataset['sentiment'], test_size=0.3, random_state=2003)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical
#AS we know for that our data is really big and hard to deal with
#so as prof told we vectorizers our data 
vectorizer = TfidfVectorizer(max_features = 2000)
X = vectorizer.fit_transform(dataset["text"]) 
Y = dataset['sentiment'] 
X_train, X_test, Y_train, Y_test = train_test_split(dataset['text'],  
                                                    dataset['sentiment'], 
                                                    test_size=0.3) 
#once again we will split our clean data into 70 and 30 train and test
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test


In [74]:
#we are using keras we are going to train our data
#importing libraries 
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
#Converts a class vector integers to binary class matrix.
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)
#our model sequential conv1d model
model = Sequential()
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr =0.01 ),
              metrics=['accuracy'])

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [81]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=30)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
109242/109242 [==============================] - 43s 398us/step - loss: 1.0078 - acc: 0.6011
Epoch 2/30
109242/109242 [==============================] - 43s 394us/step - loss: 0.9745 - acc: 0.6143
Epoch 3/30
109242/109242 [==============================] - 43s 395us/step - loss: 0.9661 - acc: 0.6193
Epoch 4/30
109242/109242 [==============================] - 43s 394us/step - loss: 0.9574 - acc: 0.6188
Epoch 5/30
109242/109242 [==============================] - 43s 394us/step - loss: 0.9446 - acc: 0.6239
Epoch 6/30
109242/109242 [==============================] - 43s 394us/step - loss: 0.9410 - acc: 0.6238
Epoch 7/30
109242/109242 [==============================] - 43s 394us/step - loss: 0.9346 - acc: 0.6274
Epoch 8/30
109242/109242 [==============================] - 43s 394us/step - loss: 0.9331 - acc: 0.6280
Epoch 9/30
109242/109242 [==============================] - 43s 397us/step - loss: 0.9329 - acc: 0.6281
Epoch 10/30
109242/109242 [==============================] - 43s

In [0]:
import pickle

with open('0889960_sentiment_Analyaze' ,'wb' ) as f:
  pickle.dump(model,f)